<a href="https://colab.research.google.com/github/jatink121/FINSEARCH_Jatin/blob/main/Q_learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Important **Libraies**

---



In [11]:
#@title

import numpy as np
import pandas as pd

file_path = '/content/drive/MyDrive/NIFTY 50_Data - NIFTY 50_Data.csv'
df = pd.read_csv(file_path)

## Reading **Data**

In [12]:
#@title
Stock_price = df['Avg']
data  = Stock_price[::-1]

## **Model with a budget constraint**

In [8]:
#@title
#Here we define state of the stocks for now i have simply chosen state to include stock price
#will update it later to consider other things too

class State :
  def __init__(self):
    self.prev_val = None
    self.next_val = None
    self.val = None
    self.avg= None
    self.volume = None
    self.q_index = None


#This is a function which randomizes the action function
def take_random_action (n, budget) :
  if budget <=12500 :
    return np.random.randint(1,2)
  if n==0 :
    return np.random.randint(0,2)
  action = np.random.randint(0,3)
  return action


#here we define the reward function for now we are choosing the reward function to be simply PnL
#Can be modified later
def get_reward(present_state,Post_state,action,buy_price,n, budget) :
  min_budget = 1000
  penalty_factor = -1
  if action==0 :                              #  buy
    PnL = (Post_state-present_state)*(n-1)
    buy_price+=present_state
    #PnL = (PnL/buy_price)*100
    n+=1
    budget-=present_state
  elif action ==2 and n !=0 :                             #sell
    PnL = (n*present_state - buy_price)
    #PnL = (PnL/buy_price) * 100
    n= 0
    buy_price = 0
    budget+=present_state*n
  else :                                   #hold
    PnL = 0
  if budget < min_budget :
    PnL = (min_budget - budget)*penalty_factor
  return PnL ,buy_price , n , budget

#Some parameters required set randomly as of now
alpha = 0.01   #learning rate
gamma = 0.999   #discount
epsilon = 0.4   #for explore-exploit trade-off
n_iterations = 1000 #number of iterations


#Now we are using Q_learning algorith which uses Q-table to predict stock price
#Creating the Q-Table and intialising it to 0
n_actions = 3      #buy,sell or hold
n_states = len(data)
q_table = np.random.rand(n_states, n_actions) * 2 - 1

#letsdefine somethings before iterating
#These n states
state_list = []
buy_price = 0
i=n_states-1
t=0
while i>0:
  temp = State()
  temp.val = data[i]
  temp.next_val = data[i-1]
  temp.q_index = t
  state_list.append(temp)
  i-=1
  t+=1
temp.val = data[0]
temp.q_index = n_states-1
state_list.append(temp)   #list of all n states

i=0
while i< n_iterations:
  epsilon-=0.0004
  buy_price = 0
  n_stocks =0
  initial_amount = 5000
  budget = 5000
  j=0
  while j< n_states-1 :
     state = state_list[j]
     if np.random.uniform() < epsilon :
      action =  take_random_action (n_stocks, budget)  # Explore (random action)
     else:
      action = np.argmax(q_table[state.q_index, :])  # Exploit (best action)
      if n_stocks == 0 :
        check = max(q_table[state.q_index, 0],q_table[state.q_index, 1])
      if check == q_table[state.q_index, 0] and budget>state.val :
        action=0
      else :
        action = 1
     reward , buy_price , n_stocks , budget = get_reward(state.val,state.next_val,action,buy_price,n_stocks , budget)
     q_table[j ,int(action)] =  (1-alpha)*q_table[j, int(action)] + \
                            alpha * (reward + gamma * np.max(q_table[j+1, :]))
     j+=1
  i+=1
print(q_table)
print(state.val*n_stocks + budget)








[[ 3.31423875e-01  6.62965541e-01  6.13861244e-01]
 [-8.61338365e-01  6.95252837e-01 -2.02952283e-01]
 [-6.38525128e-02  7.29960037e-01  2.63612389e-01]
 ...
 [-5.75165879e-01 -8.44037567e+02 -1.14066459e-01]
 [ 4.03787350e-01 -8.44441850e+02  7.05790800e-01]
 [-5.30371630e-01  2.35476501e-01  3.01057195e-01]]
12002.5


In [ ]:
returns = ((state.val*n_stocks + budget-initial_amount)/initial_amount)*100
returns

## **Now lets make a model without budget constraint**(In progress)

---



In [ ]:
#Here we define state of the stocks for now i have simply chosen state to include stock price
#will update it later to consider other things too

class State :
  def __init__(self):
    self.prev_val = None
    self.next_val = None
    self.val = None
    self.avg= None
    self.volume = None
    self.q_index = None


#This is a function which randomizes the action function
def take_random_action (n) :
  if n==0 :
    return np.random.randint(0,2)
  action = np.random.randint(0,3)
  return action


#here we define the reward function for now we are choosing the reward function to be simply PnL
#Can be modified later
def get_reward(present_state,Post_state,action,buy_price,n, budget) :
  if action==0 :                              #  buy
    PnL = (Post_state-present_state)*(n-1)
    buy_price+=present_state
    PnL = (PnL/buy_price)*100
    n+=1
    budget-=present_state
  elif action ==2 and n !=0 :                             #sell
    PnL = (n*present_state - buy_price)
    PnL = (PnL/buy_price) * 100
    n= 0
    buy_price = 0
    budget+=present_state*n
  else :                                   #hold
    PnL = 0
  return PnL ,buy_price , n , budget

#Some parameters required set randomly as of now
alpha = 0.05   #learning rate
gamma = 0.8   #discount
epsilon = 0.08    #for explore-exploit trade-off
n_iterations = 2000  #number of iterations


#Now we are using Q_learning algorith which uses Q-table to predict stock price
#Creating the Q-Table and intialising it to 0
n_actions = 3      #buy,sell or hold
n_states = len(data)
q_table = np.zeros((n_states, n_actions))

#letsdefine somethings before iterating
#These n states
state_list = []
buy_price = 0
n_stocks =0
budget = 0
i=n_states-1
t=0
while i>0:
  temp = State()
  temp.val = data[i]
  temp.next_val = data[i-1]
  temp.q_index = t
  state_list.append(temp)
  i-=1
  t+=1
temp.val = data[0]
temp.q_index = 2354
state_list.append(temp)   #list of all n states

i=0
while i< n_iterations:
  j=0
  while j< n_states-1 :
     state = state_list[j]
     if np.random.uniform() < epsilon :
      action =  take_random_action (n_stocks)  # Explore (random action)
     else:
      action = np.argmax(q_table[state.q_index, :])  # Exploit (best action)
     reward , buy_price , n_stocks , budget = get_reward(state.val,state.next_val,action,buy_price,n_stocks , budget)
     q_table[j ,action] = (1 - alpha) * q_table[j, action] + \
                            alpha * (reward + gamma * np.max(q_table[j+1, :]))
     j+=1
  i+=1
print(action)
print(q_table)







In [ ]:
#@title
#Here we define state of the stocks for now i have simply chosen state to include stock price
#will update it later to consider other things too

class State :
  def __init__(self):
    self.prev_val = None
    self.next_val = None
    self.val = None
    self.avg= None
    self.volume = None
    self.q_index = None


#This is a function which randomizes the action function
def take_random_action (n) :
  if n==0 :
      return 0
  action = np.random.randint(1,3)
  return action


#here we define the reward function for now we are choosing the reward function to be simply PnL
#Can be modified later
def get_reward(present_state,Post_state,action,buy_price,n) :
  if action==0 :                              #  buy
    PnL = (Post_state-present_state)*n
    buy_price=present_state
    n+=1
  elif action ==2 :                             #sell
    PnL = (present_state - buy_price)
    n= 0
    buy_price = 0
  else :                                   #hold
    PnL = 0
  return PnL ,buy_price , n

#Some parameters required set randomly as of now
alpha = 0.1   #learning rate
gamma = 0.9   #discount
epsilon = 0.9    #for explore-exploit trade-off
n_iterations = 500   #number of iterations


#Now we are using Q_learning algorith which uses Q-table to predict stock price
#Creating the Q-Table and intialising it to 0
n_actions = 3      #buy,sell or hold
n_states = len(data)
q_table = np.zeros((n_states, n_actions))

#letsdefine somethings before iterating
#These n states
state_list = []
buy_price = data[0]
n_stocks =1
i=0
while i< n_states-1:
  temp = State()
  temp.val = data[i]
  temp.next_val = data[i+1]
  temp.q_index = i
  state_list.append(temp)
  i+=1
temp.val = data[n_states-1]
temp.q_index = n_states-1
state_list.append(temp)   #list of all n states

i=0
while i< n_iterations:
  j=0
  while j< n_states-2 :
     state = state_list[j]
     if np.random.uniform() < epsilon :
      action =  take_random_action (n_stocks)  # Explore (random action)
     else:
      action = np.argmax(q_table[state.q_index, :])  # Exploit (best action)
      if n_stocks ==1 :
        action = np.random.randint(1,3)
     reward , buy_price , n_stocks = get_reward(state.val,state.next_val,action,buy_price,n_stocks)
     q_table[int(state.q_index) ,action] = (1 - alpha) * q_table[int(state.q_index), action] + \
                            alpha * (reward + gamma * np.max(q_table[int(state.q_index +1), :]))
     j+=1
  i+=1
print(q_table)
